We first need to do some setup so we can query the Azure API

In [1]:
import os

from azure.identity import DefaultAzureCredential
from azure.mgmt.securityinsight import SecurityInsights
from dotenv import load_dotenv


load_dotenv()


SUBSCRIPTION_ID = os.environ["SUBSCRIPTION_ID"]
RESOURCE_GROUP_NAME = os.environ["RESOURCE_GROUP_NAME"]
WORKSPACE_NAME = os.environ["WORKSPACE_NAME"]
API_VERSION = "2023-07-01-preview"

Use the Default Azure Credential (via something like `az login`). In this case we are excluding VSCode because of a bug w/ cached creds and are instead, using creds loaded as env vars from above

In [2]:
credential = DefaultAzureCredential(exclude_visual_studio_code_credential=True)

Create a Security Insights Client which is what Microsoft calls the parent interface for Sentinel Rules. Note this is scoped to a subscription.

In [3]:
security_insights_client = SecurityInsights(credential, SUBSCRIPTION_ID, api_version=API_VERSION)

Create an `AlertRulesTemplateClient` to get the alert rules. It's built from the above insights client

In [4]:
alert_rules_template_client = security_insights_client.alert_rule_templates

Get all of the Rule Templates. Note this is scoped to a tuple of `(resource group, workspace)`

In [5]:
rule_templates = alert_rules_template_client.list(RESOURCE_GROUP_NAME, WORKSPACE_NAME)


Pull down the list of ALL the built-in rule templates from Sentinel

In [6]:
example_rule_templates = list(rule_templates)
print(len(example_rule_templates))

489


Grab the first template returned for examination

In [7]:
example_rule_template = list(example_rule_templates)[0]

print(f"Name: {example_rule_template.name}")
print(f"Description : {example_rule_template.description}")
print(f"Tactics: {example_rule_template.tactics}")
print(f"Techniques: {example_rule_template.techniques}")

Name: 968358d6-6af8-49bb-aaa4-187b3067fb95
Description : This query looks for suspicious request patterns to Exchange servers that fit patterns recently
blogged about by PeterJson. This exploitation chain utilises an SSRF vulnerability in Exchange
which eventually allows the attacker to execute arbitrary Powershell on the server. In the example
powershell can be used to write an email to disk with an encoded attachment containing a shell.
Reference: https://peterjson.medium.com/reproducing-the-proxyshell-pwn2own-exploit-49743a4ea9a1
Tactics: ['InitialAccess']
Techniques: ['T1190']


Azure Sentinel Rules already included mapped ATT&CK data!